<a href="https://colab.research.google.com/github/jepilogo97/nlp/blob/main/nlp-with-gpt/nlp_with_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP con GPT

##### Jean Pierre Londoño González
##### Mini-Proyecto de clasificación de texto usando GPT
##### 21SEP2025

En este notebook harémos uso de un modelo GPT neo de 125 M que utilizaremos para generar texto a partir de un contexto inicial que proveerémos. Luego, harémos fine tuning a este modelo con un dataset de podcast en inglés del investigador de IA Lex Fridman y observar como cambia la generación de texto en función del dataset que utilicemos.

#### Referencias
- Dataset: https://huggingface.co/datasets/RamAnanth1/lex-fridman-podcasts
- https://huggingface.co/EleutherAI/gpt-neo-125m

### 1. Importación de librerias y carga de modelos

Inicio importando las librerías necesarias para el procesamiento de lenguaje natural, la manipulación de datos y la construcción del modelo. Esto incluye NumPy y pandas para el manejo y análisis de datos; Hugging Face Datasets y Transformers para la carga de corpus y la tokenización; y PyTorch junto con PyTorch Lightning para definir, entrenar y evaluar el modelo de manera estructurada.

In [1]:
import pkg_resources
import warnings

warnings.filterwarnings('ignore')

installed_packages = [package.key for package in pkg_resources.working_set]
IN_COLAB = 'google-colab' in installed_packages

/tmp/ipython-input-2396000874.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
!wget -O requirements.txt https://raw.githubusercontent.com/jepilogo97/nlp/main/nlp-with-gpt/requirements.txt
!pip install -r requirements.txt

--2025-09-22 01:07:22--  https://raw.githubusercontent.com/jepilogo97/nlp/main/nlp-with-gpt/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     228  --.-KB/s    in 0s      

2025-09-22 01:07:22 (6.72 MB/s) - ‘requirements.txt’ saved [228/228]



In [3]:
# Procesamiento de lenguaje natural y utilidades
import numpy as np  # Cálculo numérico y manejo de arreglos multidimensionales
import pandas as pd  # Manipulación y análisis de datos en estructuras tipo DataFrame

pd.set_option("display.max_rows", None)     # Todas las filas
pd.set_option("display.max_columns", None)  # Todas las columnas
pd.set_option("display.width", None)        # No cortar líneas

from datasets import Dataset, load_dataset, concatenate_datasets  # Carga y combinación de datasets de Hugging Face
from datasets.dataset_dict import DatasetDict
from collections import Counter  # Conteo de frecuencias de elementos (tokens, palabras, etc.)

import matplotlib.pyplot as plt
# Deep Learning con PyTorch
import torch  # Librería principal de tensores y operaciones en GPU/CPU
import torch.nn as nn  # Definición de capas y módulos de redes neuronales
import torch.nn.functional as F  # Funciones de activación y operaciones matemáticas de redes
from torch.utils.data import random_split, DataLoader, Subset  # Utilidades para crear y dividir datasets, cargar lotes y trabajar con subconjuntos
from torchinfo import summary

# Entrenamiento estructurado con PyTorch Lightning
from pytorch_lightning import LightningModule, Trainer  # Clase base y manejador de entrenamiento de modelos
from pytorch_lightning.loggers import TensorBoardLogger  # Registro de métricas e historial en TensorBoard
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint  # Detener entrenamiento si no mejora la métrica
from torchmetrics import Accuracy  # Métrica de precisión para clasificación supervisada

# Tipado para mayor legibilidad y validación de funciones
from typing import Optional, Tuple # Definición de tipos de datos para funciones y estructuras

from tqdm.auto import tqdm  # Barra de progreso adaptable para bucles
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments  # Tokenizador automático de modelos preentrenados de Hugging Face
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode  # Conversión de bytes a caracteres Unicode (usado en tokenización tipo GPT-2)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

# Métricas de evaluación con Scikit-learn
from sklearn.model_selection import train_test_split  # División de datos en conjuntos de entrenamiento y prueba
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métricas de evaluación de modelos de clasificación

### 2. Generative pre-training Transformer - GPT

Los modelos tipo GPT, introducidos por Radfor, et.al., de OpenAI, al igual que los modelos BERT, hacen uso extensivo de la arquitectura de transformers como hemos estado viendo. Las diferencias claves se podrían resumir en:

1. GPT utiliza bloques de **Transformer Decoder** encadenados, mientras que el modelo BERT utiliza bloques de *Transformer Encoder*
2. GPT se centra en la generación de texto basado en un contexto, la tarea principal es la predicción del siguiente token en la secuencia, mientras que BERT se centra en el completado de partes de una secuencia, en función de un contexto anterior y posterior a la secuencia de entrada. Entonces BERT se centra en la construicción de representación de lenguage, mientras que GPT se centra en la generación de texto en función de un contexto.

Sin embargo, ambos se basan en la misma premisa de pre-entrenar el modelo en tareas no-supervisadas o semi-supervisadas para que el modelo aprenda las representaciones semánticas del lenguage y luego al modelo se le pueda hacer fine tuning a tareas posteriores.

Usaremos el gpt neo en este caso de 125m, dada las limitaciones de memoria (Se intento con 2.7B y 1.3 B pero no se tenia la suficiente memoria en Ram).

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "EleutherAI/gpt-neo-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [5]:
modules = [m for m, _ in model.named_modules()]
modules

['',
 'transformer',
 'transformer.wte',
 'transformer.wpe',
 'transformer.drop',
 'transformer.h',
 'transformer.h.0',
 'transformer.h.0.ln_1',
 'transformer.h.0.attn',
 'transformer.h.0.attn.attention',
 'transformer.h.0.attn.attention.attn_dropout',
 'transformer.h.0.attn.attention.resid_dropout',
 'transformer.h.0.attn.attention.k_proj',
 'transformer.h.0.attn.attention.v_proj',
 'transformer.h.0.attn.attention.q_proj',
 'transformer.h.0.attn.attention.out_proj',
 'transformer.h.0.ln_2',
 'transformer.h.0.mlp',
 'transformer.h.0.mlp.c_fc',
 'transformer.h.0.mlp.c_proj',
 'transformer.h.0.mlp.act',
 'transformer.h.0.mlp.dropout',
 'transformer.h.1',
 'transformer.h.1.ln_1',
 'transformer.h.1.attn',
 'transformer.h.1.attn.attention',
 'transformer.h.1.attn.attention.attn_dropout',
 'transformer.h.1.attn.attention.resid_dropout',
 'transformer.h.1.attn.attention.k_proj',
 'transformer.h.1.attn.attention.v_proj',
 'transformer.h.1.attn.attention.q_proj',
 'transformer.h.1.attn.attentio

Observermos un ejemplo de generación simple.

In [6]:
text = "As part of MIT course 6S099, Artificial General Intelligence,"
best = 10

with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token, prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 14])
Dimensiones de la salida: torch.Size([1, 14, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' we': '7.82%', ' the': '6.88%', ' I': '4.46%', ' is': '3.97%', ' a': '3.79%', ' MIT': '3.69%', ' you': '3.64%', ' and': '3.04%', ' Artificial': '2.95%', ' it': '1.95%'}


### 2. Implementando una función de generación

Ahora, la idea es que este modelo nos sirva para generar texto de forma recurrente e incremental. En la última capa de los modelos tipo GPT encontrarémos un tensor con forma $(b, s, v)$, donde:

- $b$: Es el tamaño del bache, o la cantidad de secuencias a procesar.
- $s$: Es la longitud de la secuencia de entrada.
- $v$: Es el tamaño del vocabulario del modelo, cuantos tokens soporta.

Pero este es el tensor de salida, por qué tiene la forma de la secuencia de entrada?, porque cada posición en la salida corresponde a la la predicción del siguiente token de esa posición en la secuencia de entrada. En otras palabras, lo que obtenemos como predicción, es una secuencia de igual tamaño a la de entrada, movida un token hacia adelante, lo que efectivamente nos predice un solo token a la vez y por ende, el token que nos insteresa, es el último.

Lo que obtenemos en este tensor es además los logits de TODO el vocabulario del modelo, con los cuales podemos calcular las probabilidades de que cada uno sea el que continue en la secuencia. Hay varias formas de decodificar el siguiente token, la más fácil de implementar sería una decodificación codiciosa (greedy) del siguiente token, que consiste simplemente en seleccionar el token con la probabilidad más alta. Este es un enfoque simple y efectivo para algunos casos, pero al mismo tiempo sufre de poca variabilidad e incluso puede caer en generación repetitiva.

Otra opción es el muestreo, ya que justamente podemos obtener probabilidades del siguiente token, lo más lógico sería muestrear con esas opciones de probabilidad, de este modo podemos obtener mayor diversidad a la hora de generar el texto, al costo eso si de que haya mayor aleatoridad ya que se le daría la oportunidad a incluso tokens con baja probabilidad, de ser seleccionados.

Otra opción podría ser un balanceo entre una decodificación greedy y una por muestreo, en función de otro hiperparámetro que podemos definir. Esta sería una técnica muy común en el contexto de Reinforcement Learning llamade e-greedy. Se hace la aclaración de que en este ejemplo no harémos nada de RL, solamente se hace mención de esta técnica para balancear entre explotación y exploración.

In [7]:
def generate(
        model: nn.Module,
        tokenizer: PreTrainedTokenizerBase,
        start: str,
        max_length: int = 1000,
        eps: float = 0.5,
        top_n: int = 5,
        return_iterations: bool = False,
        device: str = "cpu") -> Tuple[str, Optional[pd.DataFrame]]:

    output = [start]
    iterations = []
    with torch.no_grad():
        input_ids = tokenizer(output[-1], return_tensors='pt')['input_ids'].to(device)
        for _ in range(max_length):
            # Tomamos los logits producidos por la última capa del modelo
            # Estos corresponden al siguiente token por cada posición de la cadena
            logits = model(input_ids=input_ids).logits
            # Por lo tanto, el que nos interesa es el último, que correspondería a la
            # predicción del siguiente token después del final de la cadena original
            # A este aplicamos un softmax para obtener las probabilidades por cada
            # token del vocabulario para estar presente en la cadena.
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Simplemente ordenamos por probabilidad de forma descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Utilizamos una politica tipo e-greedy para obtener el siguiente token de la secuencia
            # Un eps>=1 quiere decir que siempre se va seleccionar el token de forma 'greedy', es decir
            # siempre se toma el token con probabilidad más alta.

            # Un eps=0 quiere decir que siempre se va a muestrear el siguiente token en función
            # de las probabilidades de cada token

            # Un 0<eps<1 va a balancear de forma binomial entre tomar el token con la
            # probabilidad más alta y muestrear el token en función de sus probabilidades.
            if np.random.random_sample(1)[0] < eps:
                # Se toma el mejor token
                next_token = sorted_tokens[0].unsqueeze(dim=0)
            else:
                # Se muetrea el token de la probabilidad de distribución
                next_token = torch.multinomial(probs, 1)

            if return_iterations:
                # Mantenemos pista de todas las iteraciones para análisis
                iteration = {'input': ''.join(output)}
                best_n = sorted_tokens[:top_n].cpu().tolist()
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})' for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            output.append(tokenizer.decode(next_token))
            input_ids = torch.cat([input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        output_text = ''.join(output)
        if not return_iterations:
            return output_text, None
        else:
            df = pd.DataFrame(iterations)
            return output_text, df

Ahora observemos que pasa cuando generamos texto con nuestra función y algunos parámetros.

Primero, observemos que pasa cuando pasamos un `eps=1` que quiere decir que la generación va a ser de tipo greedy:

In [8]:
output_text, iterations_df = generate(model, tokenizer, text, max_length=15, eps=1.0, top_n=10, return_iterations=True, device=device)
print(output_text)
iterations_df.head(15)

As part of MIT course 6S099, Artificial General Intelligence, we will be using the MIT Lab to develop a new framework for AI.


,input,Choice #1,Choice #2,Choice #3,Choice #4,Choice #5,Choice #6,Choice #7,Choice #8,Choice #9,Choice #10
0,"As part of MIT course 6S099, Artificial Genera...",we (0.0782),the (0.0688),I (0.0446),is (0.0397),a (0.0379),MIT (0.0369),you (0.0364),and (0.0304),Artificial (0.0295),it (0.0195)
1,"As part of MIT course 6S099, Artificial Genera...",will (0.0905),are (0.0776),� (0.0718),have (0.0715),present (0.0243),provide (0.0195),'ll (0.0177),'ve (0.0176),'re (0.0158),use (0.0155)
2,"As part of MIT course 6S099, Artificial Genera...",be (0.1654),use (0.0468),learn (0.0461),discuss (0.0449),introduce (0.0385),explore (0.0371),cover (0.0353),show (0.0212),teach (0.0198),present (0.0170)
3,"As part of MIT course 6S099, Artificial Genera...",using (0.1106),introducing (0.0486),discussing (0.0434),working (0.0357),exploring (0.0322),presenting (0.0295),taking (0.0252),teaching (0.0231),looking (0.0224),focusing (0.0215)
4,"As part of MIT course 6S099, Artificial Genera...",the (0.1655),a (0.1058),Artificial (0.0570),our (0.0470),AI (0.0445),this (0.0442),an (0.0224),Deep (0.0147),artificial (0.0144),these (0.0127)
5,"As part of MIT course 6S099, Artificial Genera...",MIT (0.0519),latest (0.0446),same (0.0252),AI (0.0236),new (0.0234),following (0.0183),code (0.0121),Artificial (0.0120),Deep (0.0120),most (0.0080)
6,"As part of MIT course 6S099, Artificial Genera...",Lab (0.0481),Open (0.0433),Artificial (0.0347),- (0.0320),AI (0.0148),Technology (0.0144),course (0.0107),Cloud (0.0104),framework (0.0099),Learning (0.0094)
7,"As part of MIT course 6S099, Artificial Genera...",to (0.2280),� (0.1425),'s (0.0936),as (0.0927),for (0.0872),of (0.0261),at (0.0220),", (0.0206)",in (0.0166),and (0.0157)
8,"As part of MIT course 6S099, Artificial Genera...",develop (0.0766),study (0.0623),create (0.0622),build (0.0507),explore (0.0462),learn (0.0422),provide (0.0220),teach (0.0192),analyze (0.0182),help (0.0175)
9,"As part of MIT course 6S099, Artificial Genera...",a (0.2434),and (0.0764),the (0.0556),an (0.0439),AI (0.0410),our (0.0329),new (0.0290),artificial (0.0237),Artificial (0.0232),machine (0.0165)


Observamos como el input progresa a la vez que las opciones de tokens que hay. Sin importar cuantas veces invoquemos a la función con los mismos parámetros, siempre vamos a obtener los mismos resultados.

Ahora, observemos que pasa si introducimos exploración al reducir el `eps=0.5`, lo cual nos dice que aproximadamente la mitad de las veces va a elegir el siguiente token muestreando y la otra mitad explotando.

In [9]:
output_text, _ = generate(model, tokenizer, text, max_length=100, eps=0.5, device=device)
print(output_text)

As part of MIT course 6S099, Artificial General Intelligence, in the course of the MIT course Linguistics, will be elaborated. Artificial General Intelligence (AGI) is an artificial intelligence component which will be developed to solve an artificial intelligence problem and for which the AGI technology will be proven to be a superimplemented intelligent machine.

Professor Louise Garrett, Associate Professor of Hebrew and Hebrew Literature, will be responsible for answering the questions we have been asked about AI and AI-controller.

Professor Louise Garrett was appointed Academy Technical Advisor of


Lo primero observado es que aparecen extrañamente muchos signos de interrogación dentro del texto generado.
Al intentar aumentar el max length se evidencia que es demasiado costoso, pasar de 100 a 1000 toma un tiempo bastante considerable.

### 3. Generando texto con las utilidades del modelo

Ahora, la clase de Huggingface implementa la función `generate` que hace la labor de generación por nosotros, incluyendo las opciones de muestreo y explotación como hemos observado. Solo que además permite otra serie de parámetros y opciones para controlar la generación de texto.

In [10]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=100, do_sample=True, temperature=0.5, top_k=0)
print(tokenizer.decode(output[0]))

As part of MIT course 6S099, Artificial General Intelligence, the MIT Technology Review, and the MIT Technology Review, I’ve been a part of MIT’s AI and Machine Learning courses for a couple of years. I’ve also been a part of MIT’s AI and Machine Learning courses for a couple of years and have done so at MIT since 1998.

I’ve been a part of MIT’s AI and Machine Learning courses for


La exploración de la temperatura promueve creatividad y diversidad, pero con casos que evidencian incoherencia o “alucinaciones”.

### 4. Carga de dataset

Ahora, intentemos hacer fine tuning a nuestro modelo:

In [11]:
dataset = load_dataset("RamAnanth1/lex-fridman-podcasts")
dataset

DatasetDict({
    train: Dataset({
        features: ['captions', 'title'],
        num_rows: 319
    })
})

In [12]:
dataset['train'][0]

{'captions': " As part of MIT course 6S099, Artificial General Intelligence, I've gotten the chance to sit down with Max Tegmark. He is a professor here at MIT. He's a physicist, spent a large part of his career studying the mysteries of our cosmological universe. But he's also studied and delved into the beneficial possibilities and the existential risks of artificial intelligence. Amongst many other things, he is the cofounder of the Future of Life Institute, author of two books, both of which I highly recommend. First, Our Mathematical Universe. Second is Life 3.0. He's truly an out of the box thinker and a fun personality, so I really enjoy talking to him. If you'd like to see more of these videos in the future, please subscribe and also click the little bell icon to make sure you don't miss any videos. Also, Twitter, LinkedIn, agi.mit.edu if you wanna watch other lectures or conversations like this one. Better yet, go read Max's book, Life 3.0. Chapter seven on goals is my favorit

In [13]:
dataset.set_format('pandas')
df = dataset['train'].to_pandas()
df.head(10)

,captions,title
0,"As part of MIT course 6S099, Artificial Gener...",Max Tegmark: Life 3.0 | Lex Fridman Podcast #1
1,As part of MIT course 6S099 on artificial gen...,Christof Koch: Consciousness | Lex Fridman Pod...
2,"You've studied the human mind, cognition, lan...",Steven Pinker: AI in the Age of Reason | Lex F...
3,What difference between biological neural net...,Yoshua Bengio: Deep Learning | Lex Fridman Pod...
4,The following is a conversation with Vladimir...,Vladimir Vapnik: Statistical Learning | Lex Fr...
5,The following is a conversation with Guido va...,Guido van Rossum: Python | Lex Fridman Podcast #6
6,The following is a conversation with Jeff Atw...,Jeff Atwood: Stack Overflow and Coding Horror ...
7,The following is a conversation with Eric Sch...,Eric Schmidt: Google | Lex Fridman Podcast #8
8,The following is a conversation with Stuart R...,Stuart Russell: Long-Term Future of Artificial...
9,The following is a conversation with Peter Ab...,Pieter Abbeel: Deep Reinforcement Learning | L...


In [14]:
df['Palabras por podcast'] = df['captions'].str.split().apply(len)
df['Palabras por podcast'].median()

20371.0

### 5. Fine tuning

Aquí podemos observar que la mediana de longitud en terminos de palabras es de 20371. Esto es esperado, pues los podcast son largos por naturaleza.

Ahora, prepararémos el conjunto de datos para entrenamiento.

In [15]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(examples['captions'], max_length=max_len, truncation=True, padding='max_length')
    return _preprocess_function

Los modelos GPT no esperan otra cosa más que los `input_ids`, por lo que retirarémos todas las demás columnas del dataset ya que no nos son de utilidad en este momento.

In [16]:
dataset.reset_format()
tokenized_dataset = dataset['train'].map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.9)
tokenized_dataset.set_format('torch')
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 287
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 32
    })
})

Finalmente procedemos a definir el entrenamiento.

In [17]:
batch_size = 2 if IN_COLAB else 1
logging_steps = len(tokenized_dataset['train']) // batch_size
# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps,
    report_to='none'
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

In [18]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.063700,2.843010
2,2.632400,2.722750
3,2.412000,2.671136
4,2.244700,2.656195
5,2.121500,2.658655
6,2.005800,2.665448
7,1.912100,2.680070
8,1.872700,2.692358
9,1.807300,2.703142
10,1.774400,2.709343


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


CPU times: user 4min 37s, sys: 30 s, total: 5min 7s
Wall time: 9min 50s


TrainOutput(global_step=1440, training_loss=2.1821275379922653, metrics={'train_runtime': 588.9846, 'train_samples_per_second': 4.873, 'train_steps_per_second': 2.445, 'total_flos': 374832184688640.0, 'train_loss': 2.1821275379922653, 'epoch': 10.0})

### 6. Resultados

Ahora observemos los resultados.

In [19]:
output = model.generate(tokens, pad_token_id=tokenizer.eos_token_id, max_length=2000, do_sample=True, temperature=0.8)
print(tokenizer.decode(output[0]))

As part of MIT course 6S099, Artificial General Intelligence, we are studying the nature of the brain and its function in the brain and in the human mind. The brain is a computer algorithm that, at its core, is the most accessible and complete visual memory in the human brain. The idea of visual memory is that we can see our environment, our environment, our brain in a way that doesn't rely on computers at all, but is largely in the brain. Our brain is in the visual cortex where we can see images and sounds, we can read and write, and, ultimately, we can understand our world, our world's future, and in general, our world. It's the brain of the 21st century. It's the brain of the 21st century, and for that, I'd like to start by talking about artificial intelligence and artificial intelligence, especially AI that is out there, in the scientific and artificial intelligence community, that is, the AI community, and that is really useful if we can understand the technology that can solve th

Tras generar un texto de 2 000 tokens, se identificaron algunas inconsistencias, aunque el resultado es aceptable para el tamaño del modelo. No obstante, a partir de cierto punto el modelo entra en un bucle repitiendo la frase “the mind”, lo que podría deberse a patrones presentes en los datos de entrenamiento o a una configuración de decodificación poco adecuada. Este comportamiento pone de manifiesto la fuerte influencia que ejerce el conjunto de datos de entrenamiento en el desempeño final, un aspecto crucial a considerar al utilizar o realizar fine-tuning de modelos de lenguaje.

### 7. Conclusiones

#### Eficacia del flujo de análisis

- Al comparar arquitecturas de modelos de lenguaje como BERT y GPT, se identifican similitudes en su capacidad de generar representaciones ricas del lenguaje, algunas diferencias clave en su estructura y en su proceso de entrenamiento.

- Ambos modelos demuestran que un pre-entrenamiento sólido y la construcción de embeddings de alta calidad son factores críticos para alcanzar buenos resultados en tareas posteriores, evitando costos de entrenamiento desde cero.

#### Rendimiento del modelo

- Tanto BERT como GPT pueden adaptarse a una amplia variedad de tareas posteriores (clasificación, generación de texto, análisis semántico, etc.), lo que valida la versatilidad de los enfoques de transfer learning y fine tuning.

- La elección del modelo y la estrategia de entrenamiento depende de la tarea: BERT sobresale en comprensión y análisis de contexto bidireccional, mientras que GPT destaca en generación de texto coherente y fluido.

#### Limitaciones observadas

- Los modelos generativos enfrentan un dilema de exploración–explotación: una decodificación enfocada en la explotación (p. ej. greedy search) brinda mayor precisión pero tiende a producir textos monótonos; en cambio, la exploración (p. ej. sampling con temperatura alta) promueve creatividad y diversidad, pero con riesgo de incoherencia o “alucinaciones”.

- La calidad del modelo depende en gran medida de los datos de entrenamiento. Conjuntos de datos sesgados, poco representativos o de baja calidad pueden degradar el desempeño e introducir sesgos o errores difíciles de corregir.

#### Áreas de mejora

- Profundizar en la selección y curaduría de datasets, asegurando diversidad, equilibrio y relevancia para el dominio de aplicación.

- Experimentar con estrategias de decodificación y con la afinación de hiperparámetros para encontrar el punto óptimo entre creatividad, coherencia y precisión.

- Explorar técnicas de optimización y compresión que permitan desplegar modelos grandes en entornos de recursos limitados.

#### Valor práctico

- La adopción de modelos pre-entrenados como GPT ofrece una base sólida y flexible para proyectos de NLP, equilibrando costo, tiempo y calidad.

- El entendimiento de los trade-offs entre exploración y explotación, así como la adecuada selección de datos y métodos de decodificación, es esencial para alinear el modelo con los objetivos específicos del negocio y minimizar riesgos de sesgos o resultados no deseados.

### 8. Apendice

In [20]:
import pkg_resources

libs = [
    "numpy",
    "pandas",
    "datasets",
    "torch",
    "pytorch-lightning",
    "torchmetrics",
    "tqdm",
    "transformers",
    "scikit-learn"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except Exception:
        print(f"{lib}")

numpy==1.26.4
pandas==2.3.1
datasets==4.0.0
torch==2.2.0
pytorch-lightning==2.5.5
torchmetrics==1.4.0
tqdm==4.67.1
transformers==4.41.2
scikit-learn==1.6.1


In [21]:
 ## Solo correr en local

# import nbformat

## Cargar notebook
# with open("nlp_with_gpt.ipynb", "r", encoding="utf-8") as f:
    # nb = nbformat.read(f, as_version=4)

## Eliminar widgets corruptos si existen
# if "widgets" in nb["metadata"]:
    # del nb["metadata"]["widgets"]

## Guardar reparado
# with open("nlp_with_gpt.ipynb", "w", encoding="utf-8") as f:
    # nbformat.write(nb, f)